In [1]:
###########################################################
import numpy as np
random_seed = 37
np.random.seed(random_seed)
from tensorflow import set_random_seed
set_random_seed(random_seed)
###########################################################

# For each recording
#     Resample feature dataframe
#     Add first derivatives of selected features
#     Annotate frames
#     Save as new annotated dataframe, 0 => not a nod, 1 => nod

import os
import glob
import pandas as pd
import scipy.signal
import time
import pympi    # Import pympi to work with elan files

# Unified frame rate
FRAME_RATE = 30

eaf_nod_tier_name = 'Head Nods'

# Features whose first and second derivatives will be calculated
diff_selected_features = [
    ' pose_Tx', 
    ' pose_Ty', 
    ' pose_Tz', 

    ' pose_Rx', 
    ' pose_Ry', 
    ' pose_Rz',

    ' p_rx', 
    ' p_ry',
    ' p_rz'
    
    # add landmarks?
]

input_features_dir = '/home/ICT2000/jondras/datasets/vra1/listener_features'
input_annotations_dir = '/home/ICT2000/jondras/datasets/vra1/listener_nod_annotations'
output_dir = '/home/ICT2000/jondras/datasets/vra1/listener_annotated_features_perframe'

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
start_time = time.time()    
cnt = 0
for feature_file in sorted(glob.glob(input_features_dir + '/*.csv')):
    
    sid = feature_file.split('/')[-1].split('.')[0][3:]
    print(f'Processing SESID {sid}')
    
    org_df = pd.read_csv(feature_file)
#     print(org_df)
    
    # Resample feature dataframe to common frame rate, if needed
    csv_frame_rate = (org_df.shape[0] - 1) / np.sum(np.diff(org_df[' timestamp']))
    print(f'\tcsv frame rate: {csv_frame_rate}')
    if round(csv_frame_rate) == 30:
        print(f'\tNOT resampling')
        new_df = org_df.copy()
    else:
        print(f'\tREsampling')
        n_resampled_points = int(1 + (len(org_df[' timestamp']) - 1) * FRAME_RATE / csv_frame_rate)
        new_df = []
        for col_name in org_df.columns:            
            new_col = scipy.signal.resample(org_df[col_name], num=n_resampled_points)
            new_df.append(new_col)
        new_df = pd.DataFrame(np.array(new_df).T, columns=org_df.columns) 
        
    # Add first and second derivatives of selected features
    diff_features = dict()
    for feature_name in diff_selected_features:
        diff_features['diff_' + feature_name] =  np.diff(new_df[feature_name], prepend=new_df[feature_name][0])
        diff_features['diff2_' + feature_name] = np.diff(diff_features['diff_' + feature_name], 
                                                         prepend=diff_features['diff_' + feature_name][0])    
    new_df = new_df.assign(**diff_features)
            
    # Annotate frames
    # Add annotation column
    new_df = new_df.assign(nod=np.zeros(len(new_df), dtype=int))
    eaf_obj = pympi.Elan.Eaf(f'{input_annotations_dir}/SES{sid}.eaf')
    # Ignore annotations other than head nods (e.g. the default tier name)
    if eaf_nod_tier_name in eaf_obj.get_tier_names():
        anns = eaf_obj.get_annotation_data_for_tier(eaf_nod_tier_name)
        # Iterate over annotation intervals (in milliseconds)
        for ann in anns:
            interval_begin = ann[0] / 1000.
            interval_end   = ann[1] / 1000.
            new_df.nod = pd.np.where((new_df[' timestamp'] >= interval_begin) & (new_df[' timestamp'] <= interval_end), 
                                     1, new_df.nod)
#             print(interval_begin, interval_end)
#             print(np.where(np.array(new_df.nod) == 1))
    
    # Save as new annotated dataframe
#     print(new_df)
    new_df.to_csv(f'{output_dir}/SES{sid}.csv', index=False)
    cnt += 1
    print(f'Time taken: {time.time() - start_time} s\n')        
#     break

print(f'\nGenerated {cnt} annotated feature sets.')

Processing SESID 101
	csv frame rate: 29.970015322243647
	NOT resampling
Parsing unknown version of ELAN spec... This could result in errors...
Time taken: 4.662138223648071 s

Processing SESID 102
	csv frame rate: 29.969987541998563
	NOT resampling
Parsing unknown version of ELAN spec... This could result in errors...
Time taken: 11.702477931976318 s

Processing SESID 104
	csv frame rate: 29.970003273584844
	NOT resampling
Parsing unknown version of ELAN spec... This could result in errors...
Time taken: 16.734755992889404 s

Processing SESID 106
	csv frame rate: 29.9699495027727
	NOT resampling
Parsing unknown version of ELAN spec... This could result in errors...
Time taken: 22.039222240447998 s

Processing SESID 109
	csv frame rate: 29.969950130002985
	NOT resampling
Parsing unknown version of ELAN spec... This could result in errors...
Time taken: 28.271963834762573 s

Processing SESID 110
	csv frame rate: 29.970148243989478
	NOT resampling
Parsing unknown version of ELAN spec... 